In [1]:
#Initial imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, plot_confusion_matrix
from sklearn.metrics import confusion_matrix, plot_roc_curve, precision_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, StackingRegressor, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB


pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_sas('../../data/nsch_2020_topical.sas7bdat')
mask = {
    1.0: 1.0,
    2.0: 0.0
}
df['Target'] = df['K2Q35A'].map(mask)
df.dropna(subset=['Target'], inplace=True)

y = df['Target']
X = df.drop(columns=['Target', 'K2Q35A', 'K2Q35A_1_YEARS', 'K2Q35B', 'K2Q35C', 'K2Q35D', 'AUTISMMED', 'AUTISMTREAT'])

drop_cols = []
for col in X.columns:
    null_count = X[col].isna().sum()
    null_perc = X[col].isna().sum() / len(X)
    dtype = df[col].dtype
    if null_perc > 0.5:
        drop_cols.append(col)
    if dtype == 'object':
        drop_cols.append(col)

X = X.drop(columns=drop_cols)

In [2]:
!ls ..

PhilNotebooks


In [4]:
drop_cols

['FIPSST',
 'STRATUM',
 'HHID',
 'FORMTYPE',
 'K6Q41R_STILL',
 'K6Q42R_NEVER',
 'K6Q43R_NEVER',
 'K6Q13A',
 'K6Q13B',
 'K6Q14A',
 'K6Q14B',
 'K4Q28X01',
 'K4Q28X02',
 'K4Q28X03',
 'K4Q28X_EAR',
 'K4Q28X04',
 'K4Q28X05',
 'SESPLANYR',
 'SESPLANMO',
 'K4Q37',
 'SPCSERVMO',
 'LIVEUSA_YR',
 'LIVEUSA_MO',
 'A1_LIVEUSA',
 'A2_LIVEUSA',
 'HANDS',
 'COORDINATION',
 'ERRANDALONE',
 'ALLERGIES_CURR',
 'ARTHRITIS_CURR',
 'K2Q40B',
 'K2Q61B',
 'K2Q41B',
 'K2Q42B',
 'HEART_BORN',
 'HEART_CURR',
 'HEADACHE_CURR',
 'K2Q38B',
 'K2Q33B',
 'K2Q32B',
 'SICKLECELL',
 'THALASSEMIA',
 'BLOOD_OTHER',
 'BLOOD_SCREEN',
 'CYSTFIB_SCREEN',
 'GENETIC_SCREEN',
 'K2Q34B',
 'K2Q36B',
 'K2Q60B',
 'K2Q37B',
 'K2Q30B',
 'K2Q31B',
 'K2Q31D',
 'ADDTREAT',
 'SEEKCARE',
 'CONFIRMINJURY',
 'K6Q40',
 'DOCPRIVATE',
 'K6Q10',
 'K6Q12',
 'NOTELIG',
 'AVAILABLE',
 'APPOINTMENT',
 'TRANSPORTCC',
 'NOTOPEN',
 'ISSUECOST',
 'SESCURRSVC',
 'K4Q38',
 'K5Q21',
 'TREATCHILD',
 'TREATADULT',
 'MEDHISTORY',
 'WRITEPLAN',
 'RECEIVECOPY',


In [5]:
y.value_counts()

0.0    41343
1.0     1268
Name: Target, dtype: int64

1268 autistic children is likely enough to undersample instead of oversampling; which is ideal since we have many binary columns in our dataframe.

In [6]:
X.shape

(42611, 255)

In [7]:
for col in df.columns:
    null_count = df[col].isna().sum()
    null_perc = df[col].isna().sum() / len(df)
    print(f'{col} null count: {null_count}')
    print(f'{col} null percent: {null_perc}\n')

FIPSST null count: 0
FIPSST null percent: 0.0

STRATUM null count: 0
STRATUM null percent: 0.0

HHID null count: 0
HHID null percent: 0.0

FORMTYPE null count: 0
FORMTYPE null percent: 0.0

TOTKIDS_R null count: 0
TOTKIDS_R null percent: 0.0

TENURE null count: 0
TENURE null percent: 0.0

HHLANGUAGE null count: 151
HHLANGUAGE null percent: 0.0035436859027011803

SC_AGE_YEARS null count: 0
SC_AGE_YEARS null percent: 0.0

SC_SEX null count: 0
SC_SEX null percent: 0.0

K2Q35A_1_YEARS null count: 41453
K2Q35A_1_YEARS null percent: 0.9728239187064373

BIRTH_MO null count: 260
BIRTH_MO null percent: 0.0061017108258430925

BIRTH_YR null count: 379
BIRTH_YR null percent: 0.008894416934594354

MOMAGE null count: 907
MOMAGE null percent: 0.02128558353476802

K6Q41R_STILL null count: 32554
K6Q41R_STILL null percent: 0.763981131632677

K6Q42R_NEVER null count: 30586
K6Q42R_NEVER null percent: 0.717795874304757

K6Q43R_NEVER null count: 30633
K6Q43R_NEVER null percent: 0.718898875877121

K6Q13A nul

APPOINTMENT null count: 41156
APPOINTMENT null percent: 0.9658538874938396

TRANSPORTCC null count: 41156
TRANSPORTCC null percent: 0.9658538874938396

NOTOPEN null count: 41156
NOTOPEN null percent: 0.9658538874938396

ISSUECOST null count: 41156
ISSUECOST null percent: 0.9658538874938396

HOSPITALSTAY null count: 131
HOSPITALSTAY null percent: 0.003074323531482481

K6Q15 null count: 181
K6Q15 null percent: 0.004247729459529229

SESCURRSVC null count: 36282
SESCURRSVC null percent: 0.8514702776278426

K4Q36 null count: 493
K4Q36 null percent: 0.01156978245054094

K4Q38 null count: 35448
K4Q38 null percent: 0.8318978667480228

K5Q10 null count: 225
K5Q10 null percent: 0.005280326676210368

DECISIONS null count: 6940
DECISIONS null percent: 0.16286874281288868

K5Q21 null count: 22642
K5Q21 null percent: 0.5313651404566896

TREATCHILD null count: 25434
TREATCHILD null percent: 0.59688812747882

TREATADULT null count: 32464
TREATADULT null percent: 0.7618690009621929

MEDHISTORY null cou

BULLY null count: 12670
BULLY null percent: 0.2973410621670461

RECOGBEGIN null count: 35994
RECOGBEGIN null percent: 0.8447114594822933

CLEAREXP null count: 36003
CLEAREXP null percent: 0.8449226725493417

WRITENAME null count: 36009
WRITENAME null percent: 0.8450634812607073

RECSHAPES null count: 36001
RECSHAPES null percent: 0.8448757363122198

DISTRACTED null count: 35998
DISTRACTED null percent: 0.8448053319565371

WORKTOFIN null count: 35991
WORKTOFIN null percent: 0.8446410551266105

SIMPLEINST null count: 36003
SIMPLEINST null percent: 0.8449226725493417

PLAYWELL null count: 36006
PLAYWELL null percent: 0.8449930769050246

NEWACTIVITY null count: 36002
NEWACTIVITY null percent: 0.8448992044307808

HURTSAD null count: 35999
HURTSAD null percent: 0.844828800075098

CALMDOWN null count: 36011
CALMDOWN null percent: 0.8451104174978292

TEMPER null count: 36005
TEMPER null percent: 0.8449696087864635

SITSTILL null count: 36005
SITSTILL null percent: 0.8449696087864635

RECOGABC 

Target null count: 0
Target null percent: 0.0



In [8]:
X.head()

,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,BIRTH_MO,BIRTH_YR,MOMAGE,K4Q32X01,K4Q32X02,K4Q32X03,K4Q32X04,K4Q32X05,DENTALSERV1,DENTALSERV2,DENTALSERV3,DENTALSERV4,DENTALSERV5,DENTALSERV6,DENTALSERV7,K11Q43R,A1_AGE,A2_AGE,HHCOUNT,FAMCOUNT,BREATHING,SWALLOWING,STOMACH,PHYSICALPAIN,TOOTHACHES,GUMBLEED,CAVITIES,MEMORYCOND,WALKSTAIRS,DRESSING,K2Q43B,BLINDNESS,ALLERGIES,ARTHRITIS,K2Q40A,K2Q61A,K2Q41A,K2Q42A,HEART,HEADACHE,K2Q38A,K2Q33A,K2Q32A,DOWNSYN,BLOOD,CYSTFIB,GENETIC,K2Q34A,K2Q36A,K2Q60A,K2Q37A,K2Q30A,K2Q31A,CONCUSSION,K2Q05,S4Q01,OVERWEIGHT,K4Q01,USUALGO,USUALSICK,K4Q31_R,K4Q23,ALTHEALTH,K4Q27,HOSPITALSTAY,K6Q15,K4Q36,K5Q10,DECISIONS,CURRCOV,K12Q03,K12Q04,K12Q12,TRICARE,K11Q03R,HCCOVOTH,K3Q25,STOPWORK,CUTHOURS,AVOIDCHG,REPEATED,K7Q30,K7Q31,K7Q32,K7Q37,K7Q38,BORNUSA,K8Q35,EMOSUPSPO,EMOSUPFAM,EMOSUPHCP,EMOSUPWOR,EMOSUPADV,EMOSUPPEER,EMOSUPMHP,EMOSUPOTH,K9Q40,K11Q60,K11Q61,K11Q62,S9Q34,K10Q11,K10Q12,K10Q13,K10Q14,K10Q20,K10Q22,K10Q23,K9Q96,ACE3,ACE4,ACE5,ACE6,ACE7,ACE8,ACE9,ACE10,ACE12,A1_SEX,A1_BORN,A1_EMPLOYED,A1_GRADE,A1_MARITAL,A1_RELATION,A2_SEX,A2_BORN,A2_EMPLOYED,A2_GRADE,A2_MARITAL,A2_RELATION,A1_ACTIVE,A2_ACTIVE,A1_PHYSHEALTH,A1_MENTHEALTH,A2_PHYSHEALTH,A2_MENTHEALTH,K2Q01,K2Q01_D,K5Q40,K5Q41,K5Q42,K5Q43,K5Q44,K3Q20,K3Q22,K3Q21B,K6Q71_R,K7Q84_R,K7Q85_R,K7Q82_R,K7Q83_R,K7Q70_R,BULLIED_R,BULLY,TALKABOUT,WKTOSOLVE,STRENGTHS,HOPEFUL,K10Q30,K10Q31,K10Q40_R,GOFORHELP,K10Q41_R,K8Q31,K8Q32,K8Q34,HOWMUCH,ATHOMEHC,ARRANGEHC,K7Q02R_R,K7Q04R_R,PHYSACTIV,HOURSLEEP,SCREENTIME,K8Q11,FOODSIT,HCABILITY,K4Q20R,DOCROOM,WGTCONC,K4Q02_R,DENTISTVISIT,K4Q22_R,K4Q24_R,C4Q04,HOSPITALER,K4Q04_R,K5Q20_R,K5Q31_R,K8Q21,K8Q30,K7Q33,BEDTIME,K3Q04_R,ACE1,MAKEFRIEND,K4Q30_R,MENBEVCOV,YEAR,METRO_YN,MPC_YN,TOTAGE_0_5,TOTAGE_6_11,TOTAGE_12_17,TOTCSHCN,TOTNONSHCN,SC_RACE_R,SC_HISPANIC_R,SC_CSHCN,SC_ENGLISH,SC_K2Q10,SC_K2Q13,SC_K2Q16,SC_K2Q19,SC_K2Q22,SC_AGE_LT4,SC_AGE_LT6,SC_AGE_LT9,SC_AGE_LT10,AGEPOS4,TENURE_IF,TOTMALE,TOTFEMALE,SC_RACE_R_IF,SC_RACER,SC_HISPANIC_R_IF,SC_SEX_IF,BIRTHWT_OZ_S,HOUSE_GEN,FAMILY_R,CURRINS,INSGAP,INSTYPE,HIGRADE,HIGRADE_TVIS,BIRTHWT_VL,BIRTHWT_L,BIRTHWT,FPL_IF,A1_GRADE_IF,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
0,2.0,1.0,1.0,3.0,1.0,8.0,2017.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,29.0,33.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1.0,2.0,7.0,1.0,1.0,1.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,2.0,2.0,1.0,6.0,6.0,NaN,NaN,NaN,NaN,1.0,4.0,1.0,1.0,2.0,3.0,3.0,1.0,NaN,3.0,3.0,1.0,1.0,3.0,3.0,3.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,3.0,5.0,2020.0,1.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,92.0,3.0,1.0,1.0,1.0,2.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,NaN,0.0,400.0,400.0,400.0,400.0,400.0,400.0,3296.080092
1,1.0,1.0,1.0,14.0,2.0,2.0,2006.0,31.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,43.0,NaN,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,7.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,8.0,1.0,NaN,2.0,2.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0,6.0,3.0,1.0,2.0,4.0,5.0,2.0,1.0,3.0,2.0,2.0,3.0,1.0,2.0,1.0,3

In [9]:
X.shape

(42611, 255)

In [10]:
df.describe()

,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,BIRTH_MO,BIRTH_YR,MOMAGE,K6Q41R_STILL,K6Q42R_NEVER,K6Q43R_NEVER,K6Q13A,K6Q13B,K6Q14A,K6Q14B,K4Q32X01,K4Q32X02,K4Q32X03,K4Q32X04,K4Q32X05,DENTALSERV1,DENTALSERV2,DENTALSERV3,DENTALSERV4,DENTALSERV5,DENTALSERV6,DENTALSERV7,K4Q28X01,K4Q28X02,K4Q28X03,K4Q28X_EAR,K4Q28X04,K4Q28X05,SESPLANYR,SESPLANMO,K4Q37,SPCSERVMO,LIVEUSA_YR,LIVEUSA_MO,K11Q43R,A1_AGE,A1_LIVEUSA,A2_AGE,A2_LIVEUSA,HHCOUNT,FAMCOUNT,BREATHING,SWALLOWING,STOMACH,PHYSICALPAIN,HANDS,COORDINATION,TOOTHACHES,GUMBLEED,CAVITIES,MEMORYCOND,WALKSTAIRS,DRESSING,ERRANDALONE,K2Q43B,BLINDNESS,ALLERGIES,ALLERGIES_CURR,ARTHRITIS,ARTHRITIS_CURR,K2Q40A,K2Q40B,K2Q61A,K2Q61B,K2Q41A,K2Q41B,K2Q42A,K2Q42B,HEART,HEART_BORN,HEART_CURR,HEADACHE,HEADACHE_CURR,K2Q38A,K2Q38B,K2Q33A,K2Q33B,K2Q32A,K2Q32B,DOWNSYN,BLOOD,SICKLECELL,THALASSEMIA,BLOOD_OTHER,BLOOD_SCREEN,CYSTFIB,CYSTFIB_SCREEN,GENETIC,GENETIC_SCREEN,K2Q34A,K2Q34B,K2Q36A,K2Q36B,K2Q60A,K2Q60B,K2Q37A,K2Q37B,K2Q30A,K2Q30B,K2Q35A,K2Q35B,AUTISMMED,AUTISMTREAT,K2Q31A,K2Q31B,K2Q31D,ADDTREAT,CONCUSSION,SEEKCARE,CONFIRMINJURY,K2Q05,K6Q40,S4Q01,DOCPRIVATE,OVERWEIGHT,K6Q10,K6Q12,K4Q01,USUALGO,USUALSICK,K4Q31_R,K4Q23,ALTHEALTH,K4Q27,NOTELIG,AVAILABLE,APPOINTMENT,TRANSPORTCC,NOTOPEN,ISSUECOST,HOSPITALSTAY,K6Q15,SESCURRSVC,K4Q36,K4Q38,K5Q10,DECISIONS,K5Q21,TREATCHILD,TREATADULT,MEDHISTORY,WRITEPLAN,RECEIVECOPY,HEALTHKNOW,KEEPINSADULT,K12Q01_A,K12Q01_B,K12Q01_C,K12Q01_D,K12Q01_E,K12Q01_F,K12Q01_G,CURRCOV,K12Q03,K12Q04,K12Q12,TRICARE,K11Q03R,HCCOVOTH,K3Q25,STOPWORK,CUTHOURS,AVOIDCHG,ONEWORD,TWOWORDS,THREEWORDS,ASKQUESTION,ASKQUESTION2,TELLSTORY,UNDERSTAND,DIRECTIONS,POINT,DIRECTIONS2,UNDERSTAND2,RHYMEWORD,REPEATED,K7Q30,K7Q31,K7Q32,K7Q37,K7Q38,BORNUSA,K8Q35,EMOSUPSPO,EMOSUPFAM,EMOSUPHCP,EMOSUPWOR,EMOSUPADV,EMOSUPPEER,EMOSUPMHP,EMOSUPOTH,K6Q20,K6Q27,K9Q40,K9Q41,K11Q60,K11Q61,K11Q62,S9Q34,K10Q11,K10Q12,K10Q13,K10Q14,K10Q20,K10Q22,K10Q23,K9Q96,ACE3,ACE4,ACE5,ACE6,ACE7,ACE8,ACE9,ACE10,ACE12,A1_DEPLSTAT,A2_DEPLSTAT,ALLERGIES_DESC,ARTHRITIS_DESC,K2Q40C,CERPALS_DESC,K2Q41C,K2Q42C,HEART_DESC,HEADACHE_DESC,K2Q38C,K2Q33C,K2Q32C,BLOOD_DESC,GENETIC_DESC,K2Q34C,K2Q36C,K2Q60C,K2Q37C,K2Q30C,K2Q35C,K2Q31C,A1_SEX,A1_BORN,A1_EMPLOYED,A1_GRADE,A1_MARITAL,A1_RELATION,A2_SEX,A2_BORN,A2_EMPLOYED,A2_GRADE,A2_MARITAL,A2_RELATION,A1_ACTIVE,A2_ACTIVE,A1_PHYSHEALTH,A1_MENTHEALTH,A2_PHYSHEALTH,A2_MENTHEALTH,K2Q01,K2Q01_D,K5Q40,K5Q41,K5Q42,K5Q43,K5Q44,DISCUSSOPT,RAISECONC,BESTFORCHILD,K3Q20,K3Q22,K3Q21B,K6Q70_R,K6Q73_R,K6Q71_R,K6Q72_R,K7Q84_R,K7Q85_R,K7Q82_R,K7Q83_R,K7Q70_R,BULLIED_R,BULLY,RECOGBEGIN,CLEAREXP,WRITENAME,RECSHAPES,DISTRACTED,WORKTOFIN,SIMPLEINST,PLAYWELL,NEWACTIVITY,HURTSAD,CALMDOWN,TEMPER,SITSTILL,RECOGABC,TALKABOUT,WKTOSOLVE,STRENGTHS,HOPEFUL,K10Q30,K10Q31,K10Q40_R,GOFORHELP,K10Q41_R,K8Q31,K8Q32,K8Q34,HOWMUCH,ATHOMEHC,ARRANGEHC,K7Q02R_R,K7Q04R_R,PHYSACTIV,HOURSLEEP05,HOURSLEEP,SCREENTIME,K6Q60_R,K6Q61_R,K8Q11,FOODSIT,POSCHOICE,GAINSKILLS,CHANGEAGE,K2Q35D,HCABILITY,HCEXTENT,K4Q20R,DOCROOM,WGTCONC,K4Q02_R,DENTISTVISIT,K4Q22_R,TREATNEED,K4Q24_R,K4Q26,C4Q04,HOSPITALER,K4Q04_R,K5Q11,K5Q20_R,K5Q22,K5Q30,K5Q32,K5Q31_R,K8Q21,K8Q30,COUNTTO,K7Q33,BEDTIME,K3Q04_R,K6Q08_R,CONFIDENT,ACE1,USEPENCIL,MAKEFRIEND,SLEEPPOS,COLOR,K4Q30_R,STARTSCHOOL,MENBEVCOV,PLANNEEDS_R,YEAR,CBSAFP_YN,METRO_YN,MPC_YN,TOTAGE_0_5,TOTAGE_6_11,TOTAGE_12_17,TOTCSHCN,TOTNONSHCN,SC_RACE_R,SC_HISPANIC_R,SC_CSHCN,SC_ENGLISH,SC_K2Q10,SC_K2Q11,SC_K2Q12,SC_K2Q13,SC_K2Q14,SC_K2Q15,SC_K2Q16,SC_K2Q17,SC_K2Q18,SC_K2Q19,SC_K2Q20,SC_K2Q21,SC_K2Q22,SC_K2Q23,SC_AGE_LT4,SC_AGE_LT6,SC_AGE_LT9,SC_AGE_LT10,AGEPOS4,TENURE_IF,TOTMALE,TOTFEMALE,SC_RACE_R_IF,SC_RACER,SC_RACEASIA,SC_RACEAIAN,SC_HISPANIC_R_IF,SC_SEX_IF,BIRTHWT_OZ_S,BREASTFEDEND_DAY_S,BREASTFEDEND_WK_S,BREASTFEDEND_MO_S,FRSTFORMULA_DAY_S,FRSTFORMULA_WK_S,FRSTFORMULA_MO_S,FRSTSOLIDS_DAY_S,FRSTSOLIDS_WK_S,FRSTSOLIDS_MO_S,HOUSE_GEN,FAMILY_R,CURRINS,INSGAP,INSTYPE,HIGRADE,HIGRADE_TVIS,BIRTHWT_VL,BIRTHWT_L,BIRTHWT,FPL_IF,A1_GRADE_IF,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,